### Testowanie uczenia się DFA z pomocą systemu przepisywania więzów

In [1]:
import copy, sys
import cProfile

sys.path.append("../")
sys.path.append("../inferring")
sys.path.append("../utils")
sys.path.append("../utils/DFA")
from importlib import reload

import inferring.Inferring as Inferring, inferring.InferringDFA as InferringDFA,  inferring.InferringconvDFA as InferringconvDFA
import utils.automats.DFA, utils.automats.DFA.PatternDFA as PatternDFA
import utils.oracles.SRS as SRS, utils.oracles.SRSconv as SRSconv
reload(SRS)
reload(SRSconv)

reload(Inferring)
reload(InferringDFA)
reload(InferringconvDFA)

reload(utils.automats.DFA)
reload(PatternDFA)

from inferring.Inferring import Inferring
from inferring.InferringconvDFA import InferringconvDFA
from utils.automats.DFA.convDFA import convDFA
from utils.automats.DFA.DFA import DFA
from utils.automats.DFA.PatternDFA import PatternDFA
from utils.oracles.SRSconv import SRSconv

from inferring.InferringDFA import InferringDFA

In [2]:
def run_learning_process(test, oracle=None, check_consistency=False, debug=False):
    _dfa = copy.deepcopy(test.dfa)
    if test.type=="conv":
        learn_dfa = InferringconvDFA(_dfa, oracle, check_consistency=check_consistency, debug=debug)
    if test.type=="dfa":
        learn_dfa = InferringDFA(_dfa, oracle, check_consistency=check_consistency, debug=debug)
        
    dfa, cnt, cnt_ex = learn_dfa.run(counterexamples=True)
    print(f"kontrprzykłady: {cnt_ex}, cnt = {cnt}")
    return copy.deepcopy(dfa) 

In [3]:
class test_example():
    def __init__(self, type, dfa, description=''):
        self.type = type
        self.dfa = copy.deepcopy(dfa)
        self.description = description

In [4]:
test_examples = [
    test_example(type = "conv", 
                 dfa = convDFA(type="conv", 
                               dfa1 = DFA(Q=4, input_signs=['a', 'b'], δ = {(0, 'a'):1, (0, 'b'):0, (1, 'a'):1, (1, 'b'):2, (2, 'a'):3, (2, 'b'):0, (3, 'a'):3, (3, 'b'):3}, F=set([3])),
                               dfa2 = DFA(Q=4, input_signs=['a', 'b'], δ = {(0, 'a'):0, (0, 'b'):1, (1, 'a'):0, (1, 'b'):2, (2, 'a'):0, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3}, F=set([3]))),
                 description = "splot, dfa1 - wzorzec 'aba', dfa2 - wzorzec 'bbb'"),
    
    test_example(type="conv", 
                 dfa = convDFA(type="conv", 
                               dfa1=DFA(Q=3, input_signs=['a', 'b'], δ={(0,'a'):0, (0,'b'):1, (1, 'a'):0, (1, 'b'):2, (2, 'a'):2, (2, 'b'):2}, F= set([2])), 
                               dfa2=DFA(Q=3, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):0, (1, 'a'):2, (1, 'b'):0, (2, 'a'):2, (2, 'b'):2}, F= set([2]))),
                description="splot, dfa1 - wzorzec - 'bb', dfa2 - wzorzec 'aa'" ),
    
    test_example(type="conv", 
                 dfa = convDFA(type="conv", 
                               dfa1=DFA(Q=4, input_signs=['a', 'b'], δ={(0,'a'):0, (0,'b'):1, (1, 'a'):0, (1, 'b'):2, (2, 'a'):0, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3}, F= set([3])),
                               dfa2=DFA(Q=4, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):0, (1, 'a'):2, (1, 'b'):0, (2, 'a'):3, (2, 'b'):0, (3, 'a'):3, (3, 'b'):3}, F= set([3]))),
                 description="splot, dfa1 - wzorzec 'bbb', dfa2 - wzorzec 'aaa'"),
                 
    test_example(type="conv",
                 dfa=convDFA(type="conv",
                             dfa1=DFA(Q=2, input_signs=['a', 'b'], δ = {(0,'a'):1, (0, 'b'):0, (1, 'a'):1, (1, 'b'):1}, F=set([1])),
                             dfa2=DFA(Q=2, input_signs=['a', 'b'], δ = {(0,'a'):1, (0, 'b'):0, (1, 'a'):1, (1, 'b'):1}, F=set([1]))),
                description="splot, dfa1 - słowa zawierające literke 'a', dfa2 - słowa zawierające literke 'a'"),
                
    test_example(type="conv", 
                 dfa=convDFA(type="conv",
                             dfa1=DFA(Q=4, input_signs=['a', 'b'], δ={(0, 'a'):2, (0,'b'):1, (1, 'b'):1, (1, 'a'):3, (2, 'a'):2, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3}, F=set([3])),
                             dfa2=DFA(Q=4, input_signs=['a', 'b'], δ={(0, 'a'):2, (0,'b'):1, (1, 'b'):1, (1, 'a'):3, (2, 'a'):2, (2, 'b'):3, (3, 'a'):3, (3, 'b'):3}, F=set([3]))),
                description="splot, dfa1 - słowa zawierające i literę 'a' i literę 'b', dfa2 - słowa zawierające i literę 'a' i literę 'b'"),
                
    test_example(type="dfa", 
                 dfa=DFA(Q=3, input_signs=['0', '1', '2', '3', '4', '5', '6', '7'], δ={(0,'0'):0, (0,'1'):2, (0,'2'):2, (0,'3'):0, (0,'4'):2, (0,'5'):0, (0,'6'):1, (0,'7'):2, (1,'0'):2, (1,'1'):0, (1,'2'):1, (1,'3'):2, (1,'4'):1, (1,'5'):2, (1,'6'):2, (1,'7'):1, (2,'0'):2, (2,'1'):2, (2,'2'):2, (2,'3'):2, (2,'4'):2, (2,'5'):2, (2,'6'):2, (2,'7'):2}, F=set([0])),
                 description="zakodowane dodawania w systemie dwójkowym"), 
    
    test_example(type="conv", 
                 dfa=convDFA(type="conv", 
                             dfa1=DFA(Q=5, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):3, (1,'a'):2, (1,'b'):3, (2,'a'):2, (2,'b'):2, (3,'a'):1, (3,'b'):4, (4,'a'):4, (4,'b'):4}, F=set([2, 4])),
                             dfa2=DFA(Q=5, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):3, (1,'a'):2, (1,'b'):3, (2,'a'):2, (2,'b'):2, (3,'a'):1, (3,'b'):4, (4,'a'):4, (4,'b'):4}, F=set([2, 4])),
                description="splot, dfa1 - 2 wzorce 'aa' lub 'bb', dfa2 - 2 wzorce 'aa' lub 'bb'")),
                
    test_example(type="conv", 
                 dfa=convDFA(type="conv", 
                             dfa1=DFA(Q=9, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):3, (1,'a'):2, (1,'b'):3, (2,'a'):2, (2,'b'):5, (3,'a'):1, (3,'b'):4, (4,'a'):7, (4,'b'):4, (5,'a'):5, (5,'b'):6, (6,'a'):6, (6,'b'):6, (7,'a'):8, (7,'b'):7, (8,'a'):8, (8,'b'):8}, F=set([6, 8])),
                             dfa2=DFA(Q=9, input_signs=['a', 'b'], δ={(0,'a'):1, (0,'b'):3, (1,'a'):2, (1,'b'):3, (2,'a'):2, (2,'b'):5, (3,'a'):1, (3,'b'):4, (4,'a'):7, (4,'b'):4, (5,'a'):5, (5,'b'):6, (6,'a'):6, (6,'b'):6, (7,'a'):8, (7,'b'):7, (8,'a'):8, (8,'b'):8}, F=set([6, 8]))),
                description="splot, dfa1 - 2 wzorce 'aa' i 'bb', dfa2 - 2 wzorce 'aa' i 'bb'"), 
    test_example(type="conv",
                    dfa=convDFA(type="conv",
                                dfa1=PatternDFA(input_signs=['a', 'b'], patterns=['aa', 'bb']),
                                dfa2=PatternDFA(input_signs=['a', 'b', 'c'], patterns=['aa', 'bb', 'c'])),
                    description="splot, dfa1 - 2 wzorce: 'aa', 'bb', dfa2 - 3 wzorce: 'aa', 'bb', 'c'")]

In [ ]:
test = test_examples[8] 
print(test.description)
d = run_learning_process(copy.deepcopy(test), oracle=None, check_consistency=False)
d = run_learning_process(copy.deepcopy(test), oracle=SRSconv(), check_consistency=False)
d = run_learning_process(copy.deepcopy(test), oracle=SRSconv(), check_consistency=True)

### Duże automaty - ropoznające kilkadziesiąt wzorców (docelowo)

In [5]:
import random 
def random_patterns(input_signs, n=5, max_length=10):
    patterns = []
    for _ in range(n):
        p = ''.join(random.choices(population=input_signs,k=random.randint(1, max_length)))
        patterns.append(p)
    return patterns

random_patterns(input_signs=['a', 'b', 'c', 'd'], max_length=8)

['ddabdd', 'accbbacc', 'b', 'aaaaadc', 'c']

In [6]:
random.seed(25)

input_signs = ['a', 'b', 'c', 'd']
number_of_patterns = 4
max_pattern_length = 6 

patterns1, patterns2 = random_patterns(input_signs, number_of_patterns, max_pattern_length), random_patterns(input_signs, number_of_patterns, max_pattern_length)

dfa1 = PatternDFA(input_signs=input_signs, patterns=patterns1)
dfa2 = PatternDFA(input_signs=input_signs, patterns=patterns2)

print(f"patterns1 = {patterns1}")
print(f"|Q1| = {dfa1.Q}")

print(f"patterns2 = {patterns2}")
print(f"|Q2| = {dfa2.Q}")

d1 = run_learning_process(test_example(type="dfa", dfa=copy.deepcopy(dfa1)))
d2 = run_learning_process(test_example(type="dfa", dfa=copy.deepcopy(dfa2)))

print(f"after learning |Q1| = {d1.Q}")
s1 = ''.join(patterns1)
ans1 = d1.route(s1)

s2 = ''.join(patterns2)
print(f"after learning |Q2| = {d2.Q}")
ans2 = d2.route(s2)

print(f"{ans1}, {ans2}")

patterns1 = ['ddad', 'cad', 'aca', 'c']
|Q1| = 160
patterns2 = ['dcdcda', 'ccd', 'ccbbcc', 'a']
|Q2| = 392
iteracja nr: 1
iteracja nr: 2
iteracja nr: 3
iteracja nr: 4
iteracja nr: 5
kontrprzykłady: [7, 8, 9, 11], cnt = [2543, 5]
iteracja nr: 1
iteracja nr: 2
iteracja nr: 3
iteracja nr: 4
iteracja nr: 5
iteracja nr: 6
iteracja nr: 7
kontrprzykłady: [12, 13, 14, 15, 16, 16], cnt = [6646, 7]
after learning |Q1| = 36
after learning |Q2| = 52
('ddadcadacac', 1), ('dcdcdaccdccbbcca', 1)


In [7]:
dfa_ = convDFA(type="conv", dfa1=copy.deepcopy(d1), dfa2=copy.deepcopy(d2))

dfa_.route(s1+s2.upper())
print(f"|Q_conv(d1,d2)| = {dfa_.Q}")

test = test_example(type="conv", dfa=dfa_, description=f"splot, dfa1 - wzorce: {patterns1}, dfa2 - wzorce: {patterns2}")
print(test.description)

|Q_conv(d1,d2)| = 1872
splot, dfa1 - wzorce: ['ddad', 'cad', 'aca', 'c'], dfa2 - wzorce: ['dcdcda', 'ccd', 'ccbbcc', 'a']


In [8]:
cProfile.run('run_learning_process(test_example(type="dfa", dfa=copy.deepcopy(dfa1)))', sort='time')

iteracja nr: 1
iteracja nr: 2
iteracja nr: 3
iteracja nr: 4
iteracja nr: 5
kontrprzykłady: [7, 8, 9, 11], cnt = [2543, 5]
         119407 function calls (103548 primitive calls) in 0.079 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
 15842/13    0.026    0.000    0.053    0.004 copy.py:128(deepcopy)
     3648    0.009    0.000    0.030    0.000 copy.py:210(_deepcopy_tuple)
     2543    0.008    0.000    0.009    0.000 DFA.py:39(route)
    31706    0.004    0.000    0.004    0.000 {method 'get' of 'dict' objects}
     3648    0.004    0.000    0.021    0.000 copy.py:211(<listcomp>)
     30/9    0.004    0.000    0.052    0.006 copy.py:227(_deepcopy_dict)
       40    0.003    0.000    0.004    0.000 Inferring.py:166(_closed)
    19675    0.003    0.000    0.003    0.000 {built-in method builtins.id}
     3335    0.003    0.000    0.011    0.000 InferringDFA.py:32(_query_type1)
      400    0.002    0.000    0.003    0.000 

In [9]:
cProfile.run('run_learning_process(test, oracle=SRSconv(), check_consistency=True, debug=False)', sort='time')
# d = run_learning_process(test, oracle=SRSconv(), check_consistency=True, debug=True)

# do przeczytania, coś biinarnych stringach 
# https://stackoverflow.com/questions/7755082/how-to-define-a-binary-string-in-python-in-a-way-that-works-with-both-py2-and-py

iteracja nr: 1
iteracja nr: 2
iteracja nr: 3
iteracja nr: 4
iteracja nr: 5
iteracja nr: 6
iteracja nr: 7
iteracja nr: 8
iteracja nr: 9
kontrprzykłady: [19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 20, 19, 24, 24, 24, 23, 23, 23, 22, 23, 21, 22, 23, 22, 25, 22, 19, 26, 26, 26, 23, 23, 23, 24, 25, 21, 22, 25, 22, 19, 23, 23, 23, 21, 22, 22, 19, 23, 23, 23, 21, 22, 22, 19, 23, 23, 23, 21, 22, 22, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 26, 26, 26, 24, 25, 25, 21, 22, 22, 26, 26, 26, 24, 25, 25, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23], cnt = [1783235, 9]
         654505553 function calls (634314116 primitive calls) in 254.060 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(fu

In [ ]:
d = run_learning_process(copy.deepcopy(test), oracle=None, check_consistency=False)

In [ ]:
test = test_example(type="conv", dfa=dfa_, description=f"splot, dfa1 - wzorce: {patterns1}, dfa2 - wzorce: {patterns2}")
print(test.description)
d = run_learning_process(copy.deepcopy(test), oracle=None, check_consistency=False)
d = run_learning_process(copy.deepcopy(test), oracle=SRSconv(), check_consistency=False)
d = run_learning_process(copy.deepcopy(test), oracle=SRSconv(), check_consistency=True)


### Uczenie się L_{ps}'ów

In [6]:
test = test_example(type="dfa", dfa=PatternDFA(input_signs=['a', 'b', 'c', 'd'], patterns=['aa', 'b', 'c', 'd']), description="nic")

d = run_learning_process(test=test, debug=True)
d.route("abacdaba")

S = [''], rozmiar E = ['', 'a', 'b', 'c', 'd']
zamkniętość sprawdzona - S = [''], rozmiar E = ['', 'a', 'b', 'c', 'd']
hipoteza: 
(0,a) --> 0
(0,b) --> 0
(0,c) --> 0
(0,d) --> 0
stany akceptujące - set()
kontrprzyklad (z zapytania o równoważność)= aabcd



S = [''], rozmiar E = ['', 'a', 'abcd', 'b', 'bcd', 'c', 'cd', 'd']
zamkniętość sprawdzona - S = ['', 'a', 'aa', 'aab', 'aabc', 'aabcd', 'aabd'], rozmiar E = ['', 'a', 'abcd', 'b', 'bcd', 'c', 'cd', 'd']
hipoteza: 
(0,a) --> 1
(0,b) --> 0
(0,c) --> 0
(0,d) --> 0
(1,a) --> 4
(1,b) --> 0
(1,c) --> 0
(1,d) --> 0
(2,a) --> 2
(2,b) --> 2
(2,c) --> 2
(2,d) --> 3
(3,a) --> 3
(3,b) --> 3
(3,c) --> 3
(3,d) --> 3
(4,a) --> 4
(4,b) --> 6
(4,c) --> 4
(4,d) --> 4
(5,a) --> 5
(5,b) --> 5
(5,c) --> 3
(5,d) --> 5
(6,a) --> 6
(6,b) --> 6
(6,c) --> 2
(6,d) --> 5
stany akceptujące - {3}
kontrprzyklad (z zapytania o równoważność)= aacbd



S = ['', 'a', 'aa', 'aab', 'aabc', 'aabcd', 'aabd'], rozmiar E = ['', 'a', 'abcd', 'b', 'bcd', 'bd', 'c', 'cd', 'd'

('abacdaba', 0)

### Uczenie się splotu DFA - **debugowanie** 

<!-- ### Uczenie się splotów - **debugowanie**  -->

In [ ]:
import copy, sys
sys.path.append("../")
sys.path.append("../inferring")
sys.path.append("../utils")
sys.path.append("../utils/DFA")
from importlib import reload

import inferring.Inferring as Inferring, inferring.InferringDFA as InferringDFA,  inferring.InferringconvDFA as InferringconvDFA
import utils.automats.DFA
import utils.oracles.SRS as SRS, utils.oracles.SRSconv as SRSconv
reload(SRS)
reload(SRSconv)

reload(Inferring)
reload(InferringDFA)
reload(InferringconvDFA)

reload(utils.automats.DFA)


from inferring.Inferring import Inferring
from inferring.InferringconvDFA import InferringconvDFA
from utils.automats.DFA.convDFA import convDFA
from utils.automats.DFA.DFA import DFA
from utils.automats.DFA.PatternDFA import PatternDFA
from utils.oracles.SRSconv import SRSconv

from inferring.InferringDFA import InferringDFA

def run_learning_process(_dfa, oracle=None, debug=False, check_consistency=False):
    learn_dfa = InferringconvDFA(_dfa, oracle, debug=debug, check_consistency=check_consistency)
    dfa, cnt, cnt_ex = learn_dfa.run(counterexamples=True)
    print(f"kontrprzykłady: {cnt_ex}, cnt = {cnt}")
    return copy.deepcopy(dfa) 

In [ ]:
"""wzorzec - bb"""
dfa1 = DFA(Q=3, input_signs=['a', 'b'], 
           δ={(0,'a'):0, (0,'b'):1, (1, 'a'):0, (1, 'b'):2, (2, 'a'):2, (2, 'b'):2}, F= set([2]))
"""wzorzec - aa"""
dfa2 = DFA(Q=3, input_signs=['a', 'b'], 
           δ={(0,'a'):1, (0,'b'):0, (1, 'a'):2, (1, 'b'):0, (2, 'a'):2, (2, 'b'):2}, F= set([2]))

convdfa = convDFA(type='conv', dfa1=dfa1, dfa2=dfa2)
d = run_learning_process(copy.deepcopy(convdfa), oracle=SRSconv(), debug=False, check_consistency=True)

kontrprzykłady: [4, 4, 4, 4], cnt = [153, 2]
